<a href="https://colab.research.google.com/github/Aomaphisara/ULckdresearch2023/blob/main/%23LassoRegression_Feature_Importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

อ้างอิง : https://www.youtube.com/watch?v=KYHSMcP72A0

In [1]:
#library
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
import pandas as pd

# อ่านไฟล์ Excel
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Assign7: Outliers/outlier_missfr.xlsx')

# แสดงข้อมูลหัวตาราง
df.head()

,PatientUID,PatientVisitUID,Slopes,InPatient,OutPatient,Emergency,time_t1,eGFR_t1,VisitLocation,StartDTTM,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,2299,3386255,-7.158596,0,1,0,2017-10-03 09:19:54.710,63.60,เวชปฏิบัติทั่วไป (OPD)-D03,2016-08-15 08:13:26.043,...,4.9248,194.24,91.0,5.284,215.0,81.0,123.0,99.0,21.019,1
1,2299,4546471,5.550691,0,1,0,2018-12-12 09:18:49.913,70.20,เวชปฏิบัติทั่วไป (OPD)-D03,2017-10-03 07:37:22.777,...,5.1634,251.46,86.0,5.755,219.0,64.0,146.0,73.0,15.198,1
2,2299,5812902,-6.063710,0,1,0,2019-12-19 09:51:24.427,64.02,เวชปฏิบัติทั่วไป (OPD)-D03,2018-12-12 07:58:57.190,...,9.4100,313.00,100.0,5.300,193.0,62.0,129.0,65.0,30.879,1
3,2299,6947220,-12.812245,0,1,0,2021-03-04 10:01:42.113,48.54,เวชปฏิบัติทั่วไป (OPD)-D03,2019-12-19 08:19:38.647,...,4.3718,204.65,89.0,5.562,205.0,69.0,139.0,76.0,36.916,1
4,2299,8186767,28.287500,0,1,0,2022-04-21 08:45:54.580,80.47,เวชปฏิบัติทั่วไป (OPD)-D03,2021-03-04 07:55:40.703,...,5.4200,434.00,88.0,5.500,198.0,74.0,129.0,60.0,28.684,1


In [74]:
df.columns

Index(['PatientUID', 'PatientVisitUID', 'Slopes', 'InPatient', 'OutPatient',
       'Emergency', 'time_t1', 'eGFR_t1', 'VisitLocation', 'StartDTTM',
       'EndDttm', 'LabCwhen', 'G1M2-ARB', 'G3M1-Statin', 'G5M2-NSAIDs',
       'G1M5-Diuretic', 'G1M7-Beta-blocker', 'G1M3-Dihydropyridine-CCB',
       'G1M6-MRA', 'G1M1-ACEI', 'G2M2-Metformin', 'G2M3-DPP4-inhibitor',
       'G2M1-SU', 'G3M3-Fibrate', 'G2M5-SGLT2i', 'G4M1-Allopurinol',
       'G1M9-Vasodilator', 'G1M8-Alpha-blocker', 'G2M6-Insulin', 'G2M4-TZD',
       'G4M2-Febuxostat', 'G3M2-Ezetimibe', 'G1M4-Non-dihydropyridine-CCB',
       'G5M1-Ketoanalog-amino-acid', 'G2M7-GLP1-RA', 'G4M3-Uricosuric-agent',
       'Sex', 'Age', 'NATNL', 'BMI', 'BSA', 'Diastolic-BP', 'Systolic-BP',
       'D1-Hypertension', 'D2-Diabetes-mellitus', 'D3-Dyslipidemia', 'D4-SLE',
       'D5-Systemic-sclerosis', 'D6-Rheumatoid-arthritis', 'D7-HIV-infection',
       'D8-HBV-infection', 'D9-HCV-infection', 'D10-Ischemic-heart-disease',
       'D11-Heart-failu

In [75]:
df.shape

(52620, 82)

In [76]:
# เลือกคอลัมน์ที่มีชนิดข้อมูลเป็น string object
string_columns = df.select_dtypes(include='object').columns.tolist()

# แสดงรายชื่อคอลัมน์ที่เป็น string object
print(string_columns)

['time_t1', 'VisitLocation']


In [77]:
# แสดงจำนวนของค่า null ในแต่ละคอลัมน์
null_counts = df.isnull().sum()

# แสดงเฉพาะคอลัมน์ที่มีค่า null (มากกว่า 0)
null_columns = null_counts[null_counts > 0]
print(null_columns)

Series([], dtype: int64)


##slope

In [78]:
# Preview
features = df[['InPatient', 'OutPatient','Emergency','Sex', 'Age', 'NATNL', 'BMI', 'BSA', 'Diastolic-BP', 'Systolic-BP',
       'L2-BUN','L4-Sodium', 'L5-Potassium', 'L6-Chloride', 'L7-Bicarbonate', 'L8-Uric',
       'L9-Calcium', 'L10-Phosphorus', 'L11-Albumin', 'L12-Hemoglobin',
       'L13-Hematocrit', 'L14-WBC', 'L15-Platelet', 'L19-Blood-sugar',
       'L20-HbA1c', 'L21-Cholesterol', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L24-HDL-Cholesterol',
       'L28-UrineProtein-Cr-ratio',
'G1M2-ARB', 'G3M1-Statin', 'G5M2-NSAIDs', 'G1M5-Diuretic',
       'G1M7-Beta-blocker', 'G1M3-Dihydropyridine-CCB', 'G1M6-MRA',
       'G1M1-ACEI', 'G2M2-Metformin', 'G2M3-DPP4-inhibitor', 'G2M1-SU',
       'G3M3-Fibrate', 'G2M5-SGLT2i', 'G4M1-Allopurinol', 'G1M9-Vasodilator',
       'G1M8-Alpha-blocker', 'G2M6-Insulin', 'G2M4-TZD', 'G4M2-Febuxostat',
       'G3M2-Ezetimibe', 'G1M4-Non-dihydropyridine-CCB',
       'G5M1-Ketoanalog-amino-acid', 'G2M7-GLP1-RA', 'G4M3-Uricosuric-agent',
 'D1-Hypertension', 'D2-Diabetes-mellitus', 'D3-Dyslipidemia', 'D4-SLE',
       'D5-Systemic-sclerosis', 'D6-Rheumatoid-arthritis', 'D7-HIV-infection',
       'D8-HBV-infection', 'D9-HCV-infection', 'D10-Ischemic-heart-disease',
       'D11-Heart-failure', 'D12-Thalassemia', 'D13-Kidney-stone',
       'D14-Valvular-heart-disease', 'D15-Solid-malignancy',
       'D16-Hematologic-malignancy']]
target = df['Slopes']

# X and y values
X = features.values
y = target.values

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [79]:
features.shape

(52620, 70)

In [80]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

search = GridSearchCV(
    estimator=pipeline,
    param_grid={'model__alpha': np.arange(0.1, 3, 0.1)},
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=3
)
search.fit(X_train,y_train)

Fitting 5 folds for each of 29 candidates, totalling 145 fits
[CV 1/5] END ................model__alpha=0.1;, score=-89.685 total time=   0.1s
[CV 2/5] END ................model__alpha=0.1;, score=-88.639 total time=   0.2s
[CV 3/5] END ................model__alpha=0.1;, score=-93.186 total time=   0.2s
[CV 4/5] END ................model__alpha=0.1;, score=-90.346 total time=   0.1s
[CV 5/5] END ................model__alpha=0.1;, score=-89.780 total time=   0.1s
[CV 1/5] END ................model__alpha=0.2;, score=-90.330 total time=   0.1s
[CV 2/5] END ................model__alpha=0.2;, score=-89.099 total time=   0.1s
[CV 3/5] END ................model__alpha=0.2;, score=-93.776 total time=   0.2s
[CV 4/5] END ................model__alpha=0.2;, score=-90.986 total time=   0.1s
[CV 5/5] END ................model__alpha=0.2;, score=-90.641 total time=   0.1s
[CV 1/5] END model__alpha=0.30000000000000004;, score=-90.928 total time=   0.2s
[CV 2/5] END model__alpha=0.30000000000000004;,

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [81]:
search.best_params_

{'model__alpha': 0.1}

In [82]:
coef = search.best_estimator_[1].coef_
coef

array([ 3.66810154e-01, -0.00000000e+00, -0.00000000e+00,  1.24007232e-01,
        4.48010184e-01, -0.00000000e+00,  9.37741816e-02,  0.00000000e+00,
       -0.00000000e+00, -5.25564698e-01,  9.91027692e-01, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  1.51349384e+00,
        1.93505218e-01, -3.31873609e-01,  4.72814438e-01, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -9.86226961e-02, -0.00000000e+00,
       -7.42567952e-01, -0.00000000e+00, -2.28499532e-01, -3.19947579e-01,
        0.00000000e+00, -4.64435465e-01, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -6.93309823e-03,  0.00000000e+00,  0.00000000e+00,
       -3.23676838e-02, -0.00000000e+00, -1.06631808e-01, -0.00000000e+00,
       -0.00000000e+00,  9.04865725e-02,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -2.44291497e-02,
        0.00000000e+00,  

features ที่ถูกเลือกมาโดยโมเดล ที่มี taget เป็น slope

In [83]:
selected_features_slope = np.array(features.columns)[coef != 0]
selected_features_slope

array(['InPatient', 'Sex', 'Age', 'BMI', 'Systolic-BP', 'L2-BUN',
       'L8-Uric', 'L9-Calcium', 'L10-Phosphorus', 'L11-Albumin',
       'L15-Platelet', 'L20-HbA1c', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L28-UrineProtein-Cr-ratio',
       'G1M5-Diuretic', 'G1M6-MRA', 'G2M2-Metformin', 'G3M3-Fibrate',
       'G5M1-Ketoanalog-amino-acid', 'D1-Hypertension', 'D4-SLE',
       'D6-Rheumatoid-arthritis', 'D12-Thalassemia',
       'D15-Solid-malignancy'], dtype=object)

In [84]:
selected_features_slope .shape

(25,)

In [ ]:
## np.array(features.columns)[coef != 0]
#ดู feature ที่ไม่ถูกเลือก

##eGFR

In [85]:
# Preview
features = df[['InPatient', 'OutPatient','Emergency','Sex', 'Age', 'NATNL', 'BMI', 'BSA', 'Diastolic-BP', 'Systolic-BP',
       'L2-BUN','L4-Sodium', 'L5-Potassium', 'L6-Chloride', 'L7-Bicarbonate', 'L8-Uric',
       'L9-Calcium', 'L10-Phosphorus', 'L11-Albumin', 'L12-Hemoglobin',
       'L13-Hematocrit', 'L14-WBC', 'L15-Platelet', 'L19-Blood-sugar',
       'L20-HbA1c', 'L21-Cholesterol', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L24-HDL-Cholesterol',
       'L28-UrineProtein-Cr-ratio',
'G1M2-ARB', 'G3M1-Statin', 'G5M2-NSAIDs', 'G1M5-Diuretic',
       'G1M7-Beta-blocker', 'G1M3-Dihydropyridine-CCB', 'G1M6-MRA',
       'G1M1-ACEI', 'G2M2-Metformin', 'G2M3-DPP4-inhibitor', 'G2M1-SU',
       'G3M3-Fibrate', 'G2M5-SGLT2i', 'G4M1-Allopurinol', 'G1M9-Vasodilator',
       'G1M8-Alpha-blocker', 'G2M6-Insulin', 'G2M4-TZD', 'G4M2-Febuxostat',
       'G3M2-Ezetimibe', 'G1M4-Non-dihydropyridine-CCB',
       'G5M1-Ketoanalog-amino-acid', 'G2M7-GLP1-RA', 'G4M3-Uricosuric-agent',
 'D1-Hypertension', 'D2-Diabetes-mellitus', 'D3-Dyslipidemia', 'D4-SLE',
       'D5-Systemic-sclerosis', 'D6-Rheumatoid-arthritis', 'D7-HIV-infection',
       'D8-HBV-infection', 'D9-HCV-infection', 'D10-Ischemic-heart-disease',
       'D11-Heart-failure', 'D12-Thalassemia', 'D13-Kidney-stone',
       'D14-Valvular-heart-disease', 'D15-Solid-malignancy',
       'D16-Hematologic-malignancy']]
target = df['L1-eGFR']

# X and y values
X = features.values
y = target.values

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [86]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])
search = GridSearchCV(
    estimator=pipeline,
    param_grid={'model__alpha': np.arange(0.1, 3, 0.1)},
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=3
)
search.fit(X_train,y_train)

Fitting 5 folds for each of 29 candidates, totalling 145 fits
[CV 1/5] END ...............model__alpha=0.1;, score=-130.156 total time=   0.3s
[CV 2/5] END ...............model__alpha=0.1;, score=-132.527 total time=   0.3s
[CV 3/5] END ...............model__alpha=0.1;, score=-139.177 total time=   0.3s
[CV 4/5] END ...............model__alpha=0.1;, score=-133.207 total time=   0.2s
[CV 5/5] END ...............model__alpha=0.1;, score=-128.394 total time=   0.3s
[CV 1/5] END ...............model__alpha=0.2;, score=-131.413 total time=   0.2s
[CV 2/5] END ...............model__alpha=0.2;, score=-133.804 total time=   0.2s
[CV 3/5] END ...............model__alpha=0.2;, score=-140.377 total time=   0.2s
[CV 4/5] END ...............model__alpha=0.2;, score=-134.093 total time=   0.3s
[CV 5/5] END ...............model__alpha=0.2;, score=-129.642 total time=   0.2s
[CV 1/5] END model__alpha=0.30000000000000004;, score=-133.045 total time=   0.2s
[CV 2/5] END model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [87]:
search.best_params_

{'model__alpha': 0.1}

In [88]:
coef = search.best_estimator_[1].coef_
coef

array([ 6.75647862e-01, -2.43975739e-01,  0.00000000e+00, -9.33542429e-01,
       -3.62086804e+00,  0.00000000e+00,  0.00000000e+00,  9.77073152e-03,
        0.00000000e+00, -2.47579842e-02, -8.50819538e+00,  3.43189069e-01,
        0.00000000e+00, -1.66161234e-01,  2.97077270e-01, -5.77329982e+00,
        0.00000000e+00,  1.74896855e+00, -1.62573045e+00,  1.19680199e+00,
        6.55582866e-01,  1.42090904e-01,  3.70440957e-01,  5.76164527e-01,
        4.50798226e-01, -0.00000000e+00,  4.13341021e-01,  5.73157517e-01,
       -4.01953670e-01,  8.89463400e-01,  2.66514072e-01, -1.81869408e-01,
        3.75848652e-01,  3.68180107e-01, -0.00000000e+00, -5.62235254e-01,
        6.46453703e-02,  6.24100806e-01,  1.27738633e+00, -4.32383434e-01,
       -0.00000000e+00, -1.00618817e-01,  0.00000000e+00, -4.45612308e-01,
       -4.53440612e-01, -2.67767274e-01, -0.00000000e+00, -0.00000000e+00,
       -2.71369206e-01, -0.00000000e+00, -3.91262928e-01, -2.18188573e-01,
        0.00000000e+00, -

features ที่ถูกเลือกมาโดยโมเดล ที่มี taget เป็น eGFR

In [89]:
selected_features_egfr = np.array(features.columns)[coef != 0]
selected_features_egfr

array(['InPatient', 'OutPatient', 'Sex', 'Age', 'BSA', 'Systolic-BP',
       'L2-BUN', 'L4-Sodium', 'L6-Chloride', 'L7-Bicarbonate', 'L8-Uric',
       'L10-Phosphorus', 'L11-Albumin', 'L12-Hemoglobin',
       'L13-Hematocrit', 'L14-WBC', 'L15-Platelet', 'L19-Blood-sugar',
       'L20-HbA1c', 'L22-Triglyceride', 'L23-LDL-Cholesterol',
       'L24-HDL-Cholesterol', 'L28-UrineProtein-Cr-ratio', 'G1M2-ARB',
       'G3M1-Statin', 'G5M2-NSAIDs', 'G1M5-Diuretic',
       'G1M3-Dihydropyridine-CCB', 'G1M6-MRA', 'G1M1-ACEI',
       'G2M2-Metformin', 'G2M3-DPP4-inhibitor', 'G3M3-Fibrate',
       'G4M1-Allopurinol', 'G1M9-Vasodilator', 'G1M8-Alpha-blocker',
       'G4M2-Febuxostat', 'G1M4-Non-dihydropyridine-CCB',
       'G5M1-Ketoanalog-amino-acid', 'G4M3-Uricosuric-agent',
       'D2-Diabetes-mellitus', 'D4-SLE', 'D5-Systemic-sclerosis',
       'D6-Rheumatoid-arthritis', 'D9-HCV-infection', 'D11-Heart-failure',
       'D13-Kidney-stone', 'D14-Valvular-heart-disease'], dtype=object)

In [90]:
selected_features_egfr .shape

(48,)

##PCA

 Feature Importance โดยใช้วิธี Lasso Regression  จากทั้งหมด 70 feature ได้ผลดังนี้
1. ตาม slope มี 28 feature ได้แก่
       'Age', 'InPatient', 'Sex', 'G1M5-Diuretic', 'G1M6-MRA',
       'G2M2-Metformin', 'G3M3-Fibrate', 'G2M5-SGLT2i',
       'G4M1-Allopurinol', 'G4M2-Febuxostat',
       'G5M1-Ketoanalog-amino-acid', 'D1-Hypertension', 'D4-SLE',
       'D6-Rheumatoid-arthritis', 'D12-Thalassemia',
       'D15-Solid-malignancy', 'L2-BUN', 'L6-Chloride', 'L7-Bicarbonate',
       'L8-Uric', 'L9-Calcium', 'L10-Phosphorus', 'L15-Platelet',
       'L19-Blood-sugar', 'L20-HbA1c', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L28-UrineProtein-Cr-ratio'
2. ตาม eGFR มี 48 feature ได้แก่
       'Age', 'InPatient', 'OutPatient', 'Sex', 'NATNL', 'G1M2-ARB',
       'G3M1-Statin', 'G5M2-NSAIDs', 'G1M5-Diuretic',
       'G1M3-Dihydropyridine-CCB', 'G1M6-MRA', 'G1M1-ACEI',
       'G2M2-Metformin', 'G2M3-DPP4-inhibitor', 'G3M3-Fibrate',
       'G4M1-Allopurinol', 'G1M9-Vasodilator', 'G1M8-Alpha-blocker',
       'G2M4-TZD', 'G4M2-Febuxostat', 'G3M2-Ezetimibe',
       'G1M4-Non-dihydropyridine-CCB', 'G5M1-Ketoanalog-amino-acid',
       'G4M3-Uricosuric-agent', 'BMI', 'D2-Diabetes-mellitus', 'D4-SLE',
       'D6-Rheumatoid-arthritis', 'D9-HCV-infection', 'D11-Heart-failure',
       'D13-Kidney-stone', 'D14-Valvular-heart-disease', 'L2-BUN',
       'L4-Sodium', 'L7-Bicarbonate', 'L8-Uric', 'L9-Calcium',
       'L10-Phosphorus', 'L12-Hemoglobin', 'L13-Hematocrit', 'L14-WBC',
       'L15-Platelet', 'L19-Blood-sugar', 'L20-HbA1c', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L24-HDL-Cholesterol',
       'L28-UrineProtein-Cr-ratio'
3. ทำการรวม feature importance ของ slope กับ eGFR ได้ 53 feature ดังนี้
       'Age', 'InPatient', 'Sex', 'G1M5-Diuretic', 'G1M6-MRA',
       'G2M2-Metformin', 'G3M3-Fibrate', 'G2M5-SGLT2i', 'G4M1-Allopurinol',
       'G4M2-Febuxostat', 'G5M1-Ketoanalog-amino-acid', 'D1-Hypertension',
       'D4-SLE', 'D6-Rheumatoid-arthritis', 'D12-Thalassemia',
       'D15-Solid-malignancy', 'L2-BUN', 'L6-Chloride', 'L7-Bicarbonate',
       'L8-Uric', 'L9-Calcium', 'L10-Phosphorus', 'L15-Platelet',
       'L19-Blood-sugar', 'L20-HbA1c', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L28-UrineProtein-Cr-ratio', 'OutPatient',
       'NATNL', 'G1M2-ARB', 'G3M1-Statin', 'G5M2-NSAIDs',
       'G1M3-Dihydropyridine-CCB', 'G1M1-ACEI', 'G2M3-DPP4-inhibitor',
       'G1M9-Vasodilator', 'G1M8-Alpha-blocker', 'G2M4-TZD', 'G3M2-Ezetimibe',
       'G1M4-Non-dihydropyridine-CCB', 'G4M3-Uricosuric-agent', 'BMI',
       'D2-Diabetes-mellitus', 'D9-HCV-infection', 'D11-Heart-failure',
       'D13-Kidney-stone', 'D14-Valvular-heart-disease', 'L4-Sodium',
       'L12-Hemoglobin', 'L13-Hematocrit', 'L14-WBC', 'L24-HDL-Cholesterol'

In [91]:
#คอลัมน์ที่ได้จากการหา Feature Importance ของ slope 28 feature มาใช้ในการทำ PCA
slope = [ 'InPatient', 'Sex', 'Age', 'BMI', 'Systolic-BP', 'L2-BUN',
       'L8-Uric', 'L9-Calcium', 'L10-Phosphorus', 'L11-Albumin',
       'L15-Platelet', 'L20-HbA1c', 'L22-Triglyceride',
       'L23-LDL-Cholesterol', 'L28-UrineProtein-Cr-ratio',
       'G1M5-Diuretic', 'G1M6-MRA', 'G2M2-Metformin', 'G3M3-Fibrate',
       'G5M1-Ketoanalog-amino-acid', 'D1-Hypertension', 'D4-SLE',
       'D6-Rheumatoid-arthritis', 'D12-Thalassemia',
       'D15-Solid-malignancy']
slope = df[slope]
slope.head()

,InPatient,Sex,Age,BMI,Systolic-BP,L2-BUN,L8-Uric,L9-Calcium,L10-Phosphorus,L11-Albumin,...,G1M5-Diuretic,G1M6-MRA,G2M2-Metformin,G3M3-Fibrate,G5M1-Ketoanalog-amino-acid,D1-Hypertension,D4-SLE,D6-Rheumatoid-arthritis,D12-Thalassemia,D15-Solid-malignancy
0,0,1,73,20.80,127.0,18.500,4.481,9.643,3.244,4.652,...,0,0,0,0,0,0,0,0,0,0
1,0,1,74,20.27,105.0,13.000,3.800,9.629,3.787,4.729,...,0,0,0,0,0,0,0,0,0,0
2,0,1,75,20.72,138.0,6.500,4.322,8.864,3.486,3.798,...,0,0,0,0,0,0,0,0,0,0
3,0,1,76,20.27,134.0,13.873,4.528,9.248,3.615,4.390,...,35,0,0,0,0,0,0,0,0,0
4,0,1,78,19.05,103.0,14.500,5.298,9.149,3.816,4.194,...,0,0,0,0,0,1,0,0,0,0


In [92]:
#ใช้คอลัมน์ที่ได้จากการหา Feature Importance ของ eGFR 48 feature มาใช้ในการทำ PCA
egfr = ['InPatient', 'OutPatient', 'Sex', 'Age', 'BSA', 'Systolic-BP',
       'L2-BUN', 'L4-Sodium', 'L6-Chloride', 'L7-Bicarbonate', 'L8-Uric',
       'L10-Phosphorus', 'L11-Albumin', 'L12-Hemoglobin',
       'L13-Hematocrit', 'L14-WBC', 'L15-Platelet', 'L19-Blood-sugar',
       'L20-HbA1c', 'L22-Triglyceride', 'L23-LDL-Cholesterol',
       'L24-HDL-Cholesterol', 'L28-UrineProtein-Cr-ratio', 'G1M2-ARB',
       'G3M1-Statin', 'G5M2-NSAIDs', 'G1M5-Diuretic',
       'G1M3-Dihydropyridine-CCB', 'G1M6-MRA', 'G1M1-ACEI',
       'G2M2-Metformin', 'G2M3-DPP4-inhibitor', 'G3M3-Fibrate',
       'G4M1-Allopurinol', 'G1M9-Vasodilator', 'G1M8-Alpha-blocker',
       'G4M2-Febuxostat', 'G1M4-Non-dihydropyridine-CCB',
       'G5M1-Ketoanalog-amino-acid', 'G4M3-Uricosuric-agent',
       'D2-Diabetes-mellitus', 'D4-SLE', 'D5-Systemic-sclerosis',
       'D6-Rheumatoid-arthritis', 'D9-HCV-infection', 'D11-Heart-failure',
       'D13-Kidney-stone', 'D14-Valvular-heart-disease']
egfr = df[egfr]
egfr.head()

,InPatient,OutPatient,Sex,Age,BSA,Systolic-BP,L2-BUN,L4-Sodium,L6-Chloride,L7-Bicarbonate,...,G5M1-Ketoanalog-amino-acid,G4M3-Uricosuric-agent,D2-Diabetes-mellitus,D4-SLE,D5-Systemic-sclerosis,D6-Rheumatoid-arthritis,D9-HCV-infection,D11-Heart-failure,D13-Kidney-stone,D14-Valvular-heart-disease
0,0,1,1,73,1.39,127.0,18.500,142.18,103.70,26.199,...,0,0,2,0,0,0,0,0,0,0
1,0,1,1,74,1.38,105.0,13.000,141.79,102.99,26.881,...,0,0,2,0,0,0,0,0,0,0
2,0,1,1,75,1.38,138.0,6.500,143.00,103.00,29.700,...,0,0,1,0,0,0,0,0,0,0
3,0,1,1,76,1.36,134.0,13.873,141.58,102.78,27.198,...,0,0,1,0,0,0,0,0,0,0
4,0,1,1,78,1.33,103.0,14.500,141.00,102.00,27.200,...,0,0,1,0,0,0,0,0,0,0


In [93]:
# หาคอลัมน์ที่ซ้ำกัน
common_columns = list(set(slope.columns).intersection(egfr.columns))
len(common_columns)


20

In [94]:
# หาคอลัมน์ที่ไม่ซ้ำกันใน slope
unique_columns_slope = list(set(slope.columns).difference(common_columns))
len(unique_columns_slope)

5

In [95]:
# หาคอลัมน์ที่ไม่ซ้ำกันใน egfr
unique_columns_egfr = list(set(egfr.columns).difference(common_columns))
len(unique_columns_egfr)

28

In [96]:
# นำคอลัมน์ทั้งหมดมารวมกัน
selected_features = common_columns + unique_columns_slope + unique_columns_egfr

# ลำดับคอลัมน์ตาม slope
slope_columns_order = slope.columns.tolist()

# ลำดับคอลัมน์ตาม egfr
egfr_columns_order = egfr.columns.tolist()

# นำคอลัมน์ทั้งหมดมารวมกัน โดยให้ลำดับตาม slope แล้ว egfr
selected_features_order = slope_columns_order + [col for col in egfr_columns_order if col not in slope_columns_order]

# สร้าง DataFrame ที่มีเฉพาะ feature ที่เลือกและเรียงลำดับตามลำดับที่กำหนด
data = df[selected_features_order]
data.head()


,InPatient,Sex,Age,BMI,Systolic-BP,L2-BUN,L8-Uric,L9-Calcium,L10-Phosphorus,L11-Albumin,...,G1M8-Alpha-blocker,G4M2-Febuxostat,G1M4-Non-dihydropyridine-CCB,G4M3-Uricosuric-agent,D2-Diabetes-mellitus,D5-Systemic-sclerosis,D9-HCV-infection,D11-Heart-failure,D13-Kidney-stone,D14-Valvular-heart-disease
0,0,1,73,20.80,127.0,18.500,4.481,9.643,3.244,4.652,...,0,0,0,0,2,0,0,0,0,0
1,0,1,74,20.27,105.0,13.000,3.800,9.629,3.787,4.729,...,0,0,0,0,2,0,0,0,0,0
2,0,1,75,20.72,138.0,6.500,4.322,8.864,3.486,3.798,...,0,0,0,0,1,0,0,0,0,0
3,0,1,76,20.27,134.0,13.873,4.528,9.248,3.615,4.390,...,0,0,0,0,1,0,0,0,0,0
4,0,1,78,19.05,103.0,14.500,5.298,9.149,3.816,4.194,...,0,0,0,0,1,0,0,0,0,0


In [97]:
#รวม feature importance ของ slope กับ eGFR ได้ 53 คอลัมน์
# catagory [InPatient, Sex, OutPatient, NATNL]
data.columns

Index(['InPatient', 'Sex', 'Age', 'BMI', 'Systolic-BP', 'L2-BUN', 'L8-Uric',
       'L9-Calcium', 'L10-Phosphorus', 'L11-Albumin', 'L15-Platelet',
       'L20-HbA1c', 'L22-Triglyceride', 'L23-LDL-Cholesterol',
       'L28-UrineProtein-Cr-ratio', 'G1M5-Diuretic', 'G1M6-MRA',
       'G2M2-Metformin', 'G3M3-Fibrate', 'G5M1-Ketoanalog-amino-acid',
       'D1-Hypertension', 'D4-SLE', 'D6-Rheumatoid-arthritis',
       'D12-Thalassemia', 'D15-Solid-malignancy', 'OutPatient', 'BSA',
       'L4-Sodium', 'L6-Chloride', 'L7-Bicarbonate', 'L12-Hemoglobin',
       'L13-Hematocrit', 'L14-WBC', 'L19-Blood-sugar', 'L24-HDL-Cholesterol',
       'G1M2-ARB', 'G3M1-Statin', 'G5M2-NSAIDs', 'G1M3-Dihydropyridine-CCB',
       'G1M1-ACEI', 'G2M3-DPP4-inhibitor', 'G4M1-Allopurinol',
       'G1M9-Vasodilator', 'G1M8-Alpha-blocker', 'G4M2-Febuxostat',
       'G1M4-Non-dihydropyridine-CCB', 'G4M3-Uricosuric-agent',
       'D2-Diabetes-mellitus', 'D5-Systemic-sclerosis', 'D9-HCV-infection',
       'D11-Heart-failu

In [98]:
#data.to_csv('LassoRegression_FI.csv')